# Download embeddings model from hugging face

In [1]:
import sys
print(sys.executable)

/root/week_2/venv/bin/python


In [2]:

from huggingface_hub import hf_hub_download
import torch
from model import CBOW

# model withou title hackers
model_path = hf_hub_download(repo_id="cocoritzy/cbow-upvotes_model", filename="cbow_model.pt")


/root/week_2/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load(model_path, map_location=device) #A checkpoint is a file that saves the state of your model (


In [4]:
print(checkpoint["model_state_dict"])

OrderedDict([('embeddings.weight', tensor([[-6.4979e-01, -5.7941e-01,  6.6106e-01,  ..., -2.2381e+00,
         -4.7060e-04,  1.5041e-01],
        [-4.9834e-01, -1.3562e+00,  5.9971e-02,  ...,  2.2810e-02,
         -4.2601e+00,  1.6657e+00],
        [-3.5826e-01,  1.8044e+00,  3.4292e-01,  ..., -4.9925e-01,
          6.3616e-02, -2.0757e-01],
        ...,
        [ 3.0328e+00,  2.2346e+00, -4.0947e+00,  ...,  1.5389e+00,
         -3.1774e+00,  1.5126e+00],
        [ 9.5289e-01, -7.5809e-01, -1.0489e+00,  ...,  2.7495e+00,
         -2.2415e+00,  6.3896e-01],
        [ 6.0927e-01, -3.6017e-01, -2.4570e+00,  ...,  2.5984e+00,
         -1.5189e+00, -1.4678e+00]], device='cuda:0')), ('linear.weight', tensor([[-0.9190, -3.4283, -1.5246,  ...,  0.7419,  2.3657, -0.5639],
        [-0.9271, -3.3398, -1.2597,  ...,  0.3505,  2.7461, -0.6402],
        [-1.0051, -3.6519, -1.2860,  ...,  0.3628,  2.5529, -0.4234],
        ...,
        [-1.0449, -2.6130, -1.2064,  ...,  0.3532,  2.4671, -0.5399],
   

In [5]:


# Load dim
token_to_index = checkpoint["token_to_index"]

embedding_dim= checkpoint["embedding_dim"]
vocab_size = len(token_to_index)  # fill in actual size

embedding_dim = checkpoint['embedding_dim']

# Load the model architecture
model = CBOW(voc=vocab_size, emb=embedding_dim)


model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)
model.eval()
 # it contains the model's parameters and other information needed to resume training or make predictions.




CBOW(
  (embeddings): Embedding(30000, 100)
  (linear): Linear(in_features=100, out_features=30000, bias=True)
)

# Load query and title documents 

In [21]:

from datasets import load_dataset

dataset = load_dataset("cocoritzy/week_2_triplet_dataset_hard_negatives")

split_data = dataset["train"].train_test_split(test_size=0.2, seed=42) # 80% train, 20% test


df_train = split_data["train"].to_pandas()
df_train.columns


Index(['query_id', 'query', 'positive_passage', 'negative_passage',
       'negative_index_in_group'],
      dtype='object')

# Convert title to embedding

In [14]:
embedding_layer = model.embeddings

In [15]:
def title_to_embedding(title):
    tokens = title.lower().split()
    indices = [token_to_index.get(tok, 0) for tok in tokens]  # 0 for unknowns

    indices_tensor = torch.tensor(indices, dtype=torch.long, device=device)

    with torch.no_grad():
        embeds = embedding_layer(indices_tensor) # [num_tokens, embedding_dim]
        return embeds.mean(dim=0) # average pooling


In [35]:
negative = df_train.iloc[12]["negative_passage"]
postive = df_train.iloc[12]["positive_passage"]

print("nagative:",negative,"positive:",postive)

nagative: FLOT/FEBA may include covering and screening forces. The Forward Line of Enemy Troops (FLET) , is the FEBA from the enemy's perspective. The adjective variant of the term front line is used to describe materiel or personnel intended for forward use-at sea, on land or in the air-i.e. at the front line. positive: forward line of own troops (FLOT) (JP 1-02) - A line which indicates the most forward positions of friendly forces in any kind of military operation at a specific time. The FLOT normally identifies the forward location of covering and screening forces. (Army) - The FLOT may be at, beyond, or short of the FEBA. An enemy FLOT indicates the forward most position of hostile forces. (See also line of contact (LC).) See FMs 1-111, 6-20 series, 7-20, 7-30, 71-100, 71-123, 100-5, and 100-15. An area in hostile or insurgent territory which has a 360-degree defense and which supports combat patrols or larger operations with combat support and combat service support assets. (See 

In [51]:
query = df_train.iloc[12]["query"]
negative = df_train.iloc[12]["negative_passage"]
postive = df_train.iloc[12]["positive_passage"]

a = title_to_embedding(query)
b = title_to_embedding("1")
c = title_to_embedding(postive)


In [52]:
import torch.nn.functional as F

def cosine_similarity(x, y):
    return F.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0)).item()


In [53]:
sim_pos = cosine_similarity(a, b)
sim_neg = cosine_similarity(a, c)
print(sim_pos, sim_neg)

-0.00491216778755188 0.3279302716255188
